In [2]:
from pyspark import SparkContext, SparkConf
import sys
import os
import random as rand
import CountTriangles as ct
from collections import defaultdict

In [3]:
conf = SparkConf().setAppName('TriangleCountExercise')
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/28 10:13:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# 1. Read number of partitions
C = 2
data_path = "facebook_small.txt"
assert os.path.isfile(data_path), "File or folder not found"
docs = sc.textFile(data_path,minPartitions=C).cache()
docs.repartition(numPartitions=C)

MapPartitionsRDD[13] at coalesce at DirectMethodHandleAccessor.java:104

In [5]:
def CountTriangles(edges):
    # Create a defaultdict to store the neighbors of each vertex
    neighbors = defaultdict(set)
    for edge in edges:
        u, v = edge
        neighbors[u].add(v)
        neighbors[v].add(u)

    # Initialize the triangle count to zero
    triangle_count = 0

    # Iterate over each vertex in the graph.
    # To avoid duplicates, we count a triangle <u, v, w> only if u<v<w
    for u in neighbors:
        # Iterate over each pair of neighbors of u
        for v in neighbors[u]:
            if v > u:
                for w in neighbors[v]:
                    # If w is also a neighbor of u, then we have a triangle
                    if w > v and w in neighbors[u]:
                        triangle_count += 1
    # Return the total number of triangles in the graph
    return triangle_count

In [9]:
def count_triangles_partitions(pairs):
    edges = [pair[1] for pair in pairs]
    return [(DEFAULT_KEY, CountTriangles(edges))]

In [16]:
def MR_ApproxTCwithSparkPartitions(docs):
    tri_count = (
                docs.map(lambda x: (rand.randint(0, C-1), x))  # MAP PHASE (R1)
                .mapPartitions(count_triangles_partitions)  # REDUCE PHASE (R1)
                .reduceByKey(lambda x, y: x + y)  # REDUCE PHASE (R2)
        )

    print(tri_count.collect())
    return(tri_count.collect())

In [15]:
a = docs.map(lambda x: (int(x.split(",")[0]), int(x.split(",")[1]))) # <-- MAP PHASE (R1)

#(C**2)*
MR_ApproxTCwithSparkPartitions(a)

[(1, 1332868)]


[(1, 1332868)]